### GD(Gradient Descent)
- 전체 학습 데이터를 기반으로 GD 계산
- 입력 데이터가 크고 레이어가 많을 수록 GD를 계산하는데 많은 Computing 자원이 소모

### SGD(Stochastic Gradient Descent)
- 전체 학습 데이터 중 한 개만 임의로 선택하여 GD 계산
- 수렴 정확도가 상대적으로 떨어진다.

### Mini - Batch GD
- 전체 학습 데이터 중 **특정 크기 만큼**(Batch 크기) 임의로 선택해서 GD 계산


일반적으로 Mini - Batch GD가 대부분의 딥러닝 Framework에서 채택됨  
전체 학습 데이터 400건으로 할 경우, 랜덤이기 때문에 중복 될 수있다. 따라서 순차적인 Mini-Batch

# SGD 실습

In [22]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

boston = load_boston()
print(boston.keys()) 
# print(boston) # dictionary 타입으로 되어있다.

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])


In [23]:
bostonDF = pd.DataFrame(boston.data, columns = boston.feature_names)
bostonDF['PRICE'] = boston.target     # 종속 변수 추가
print(bostonDF.shape)
bostonDF.head()

(506, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

In [6]:
def get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate=0.01):
    
    # 데이터 건수
    N = target_sgd.shape[0]   # 한 개

    # 예측 값
    predicted_sgd = w1 * rm_sgd + w2  * lstat_sgd + bias   # 506개를 한 꺼번에 계산(column)
    
    # 실제 값과 예측값의 차이
    diff_sgd = target_sgd - predicted_sgd     # 잔차(Residual)
    
    bias_factor = np.ones((N,))  # 편향(bias)

    w1_update = -(2/N) * learning_rate * (np.dot(rm_sgd.T, diff_sgd))      # w1를 기준으로 편미분
    w2_update = -(2/N) * learning_rate * (np.dot(lstat_sgd.T, diff_sgd))   # w1를 기준으로 편미분
    bias_update = -(2/N) * learning_rate * (np.dot(bias_factor.T, diff_sgd))

    return bias_update, w1_update, w2_update

In [4]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs 수만큼 반복적으로 Weight와 Bias를 update 적용.
def st_gradient_descent(features, target, iter_epochs=1000, verbose=True):
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    w1 = np.zeros((1,))    # 원래는 랜덤 값으로 하는게 맞다.
    w2 = np.zeros((1,))
    bias = np.ones((1,))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 저장,호출 시 numpy array 형태로 RM과 LSTAT로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행
    for i in range(iter_epochs):
        # iteration 마다 stochapstic gradient descent 를 수행할 데이터를 한 개만 추출. 추출할 데이터의 인덱스를 random.choice() 로 선택.
        stochastic_index = np.random.choice(target.shape[0], 1)
        rm_sgd = rm[stochastic_index]
        lstat_sgd = lstat[stochastic_index]
        target_sgd = target[stochastic_index]

        # SGD 기반으로 weight/bias update 값 계산
        bias_update, w1_update, w2_update = get_update_weights_value_sgd(bias, w1, w2, rm_sgd, lstat_sgd, target_sgd, learning_rate)
        # SGD로 구한 weight/bias의 update 적용
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update
        if verbose:
            print("Epoch: ", i + 1, "/", iter_epochs)
            # Loss는 전체 학습 데이터 기반으로 구해야 한다.
            predicted = w1 * rm + w2 * lstat + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print("w1:", w1, "w2:", w2, "bias:", bias, "loss", mse_loss)

    return w1, w2, bias

In [ ]:
w1, w2, bias = st_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs=1000, verbose=True)
print(" #### 최종 w1, w2, bias #### ")
print(w1, w2, bias)

# Mini - Batch 실습
- batch size 만큼 데이터가 학습 한다.

In [15]:
def get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):
    
    # 데이터 건수
    N = target_batch.shape[0]
    # 예측 값. 
    predicted_batch = w1 * rm_batch+ w2 * lstat_batch + bias
    # 실제값과 예측값의 차이 
    diff_batch = target_batch - predicted_batch
    # bias 를 array 기반으로 구하기 위해서 설정. 
    bias_factors = np.ones((N,))
    
    # weight와 bias를 얼마나 update할 것인지를 계산.  
    w1_update = -(2/N)*learning_rate*(np.dot(rm_batch.T, diff_batch))
    w2_update = -(2/N)*learning_rate*(np.dot(lstat_batch.T, diff_batch))
    bias_update = -(2/N)*learning_rate*(np.dot(bias_factors.T, diff_batch))
    
    # Mean Squared Error값을 계산. 
    #mse_loss = np.mean(np.square(diff))
    
    # weight와 bias가 update되어야 할 값 반환 
    return bias_update, w1_update, w2_update

In [18]:
# batch_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음.
def batch_gradient_descent(features, target, iter_epochs=1000, batch_size = 30, verbose=True):
    np.random.seed = 2001
    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    w1 = np.zeros((1,))    # 원래는 랜덤 값으로 하는게 맞다.
    w2 = np.zeros((1,))
    bias = np.ones((1,))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 저장,호출 시 numpy array 형태로 RM과 LSTAT로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행
    for i in range(iter_epochs):
        print(f"현재 iteration은 {i}입니다.")
        # batch_size 만큼 데이터를 가져와서 weight/bias update를 수행하는 로직을 전체 데이터 건수만큼 반복
        for batch_step in range(0, target.shape[0], batch_size): # 0 ~ 506 을 batch_size만큼 나눠서 출력
            # batch_szie만큼 순차적인 데이터를 가져옴.
            # 0 ~ 30, 30 ~ 60, 60 ~ 90 ...
            rm_batch = rm[batch_step:batch_step + batch_size]
            lstat_batch = lstat[batch_step:batch_step + batch_size]
            target_batch = target[batch_step:batch_step + batch_size]

            # SGD 기반으로 weight/bias update 값 계산
            bias_update, w1_update, w2_update = get_update_weights_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)
            # SGD로 구한 weight/bias의 update 적용
            w1 = w1 - w1_update
            w2 = w2 - w2_update
            bias = bias - bias_update
            if verbose:
                print("Epoch :", i + 1, "/", iter_epochs, 'batch step:', batch_step)
                # Loss는 전체 학습 데이터 기반으로 구해야 한다.
                predicted = w1 * rm + w2 * lstat + bias
                diff = target - predicted
                mse_loss = np.mean(np.square(diff))
                print("w1:", w1, "w2:", w2, "bias:", bias, "loss", mse_loss)

    return w1, w2, bias

In [ ]:
w1, w2, bias = batch_gradient_descent(scaled_features, bostonDF['PRICE'].values, iter_epochs = 5000, batch_size=30, verbose=True)
print("##### 최종 w1, w2, bias ######")
print(w1, w2, bias)

### 참고 예시

In [5]:
batch_indexes = np.random.choice(506, 30)
print(batch_indexes)

# random하게 뽑은 인덱스로 RM값 추출
bostonDF['RM'].values[batch_indexes]

[382  83 482 142 269 417  83  44 232 294 259 436 381 224 117 407  94 237
  77 343 241   4   8 233 190 370 237 343 283 443]


array([5.536, 6.167, 7.061, 5.403, 5.92 , 5.304, 6.167, 6.069, 8.337,
       6.009, 6.842, 6.461, 6.545, 8.266, 6.021, 5.608, 6.249, 7.358,
       6.14 , 6.696, 6.095, 7.147, 5.631, 8.247, 6.951, 7.016, 7.358,
       6.696, 7.923, 6.485])

In [ ]:
for batch_step in range(0, 506, 30):
    print(batch_step)

In [7]:
# 이렇게 되면 오류가 발생한다.
bostonDF['PRICE'].values[507]

IndexError: ignored

In [8]:
# 최대한 찾을 수 있는 부분까지 찾아준다.
bostonDF['PRICE'].values[480:510]

array([23. , 23.7, 25. , 21.8, 20.6, 21.2, 19.1, 20.6, 15.2,  7. ,  8.1,
       13.6, 20.1, 21.8, 24.5, 23.1, 19.7, 18.3, 21.2, 17.5, 16.8, 22.4,
       20.6, 23.9, 22. , 11.9])

# keras로 작업하기

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    # 단 하나의 units 설정. input_shape는 2차원, 회귀이므로 activation은 설정하지 않음. 
    # weight와 bias 초기화는 kernel_inbitializer와 bias_initializer를 이용. 
    Dense(1, input_shape=(2, ), activation=None, kernel_initializer='zeros', bias_initializer='ones')
])
# Adam optimizer를 이용하고 Loss 함수는 Mean Squared Error, 성능 측정 역시 MSE를 이용하여 학습 수행. 
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['mse'])

# Keras는 반드시 Batch GD를 적용함. batch_size가 None이면 32를 할당. 
model.fit(scaled_features, bostonDF['PRICE'].values, batch_size=30, epochs=1000)